## CNN (Convolutional Neural Network)

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from helper_functions import accuracy_fn
from tqdm.auto import tqdm

from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 2.0.0+cu117
torchvision version: 0.15.1+cu117


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_data = FashionMNIST(
    root="data",
    download=True,
    train=True,
    transform=ToTensor()
)
test_data = FashionMNIST(
    root="data",
    download=True,
    train=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(train_data, 32, shuffle=True)
test_dataloader = DataLoader(test_data, 32, shuffle=True)

In [4]:
classes = train_data.classes

In [5]:
class CNNModel(nn.Module):
    def __init__(self, input, hidden, output, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        self.block1 = nn.Sequential(
            nn.Conv2d(input, hidden, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden, hidden, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(hidden, hidden, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden, hidden, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden*7*7,
                      out_features=output)
        )
        
    def forward(self, x):
        return self.classifier(self.block2(self.block1(x)))


In [6]:
model = CNNModel(1, 10, len(classes)).to(device)

In [7]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    for (X, y) in data_loader:
        # Send data to GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")


def test_step(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
        
        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")


def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn,
               device):
    """Returns a dictionary containing the results of model predicting on data_loader.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            # Make predictions with the model
            y_pred = model(X)
            
            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, 
                                y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)
        
        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)
        
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1)

In [9]:
epochs = 3
for epoch in tqdm(range(epochs)):
    print(f"Epoch {epoch+1} of {epochs}")
    train_step(model, train_dataloader, loss_fn, optimizer, accuracy_fn, device)
    test_step(model, train_dataloader, loss_fn, accuracy_fn, device)
    
    

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1 of 3
Train loss: 0.58543 | Train accuracy: 78.58%
Test loss: 0.36163 | Test accuracy: 87.28%

Epoch 2 of 3
Train loss: 0.35838 | Train accuracy: 86.92%
Test loss: 0.33414 | Test accuracy: 88.59%

Epoch 3 of 3
Train loss: 0.32199 | Train accuracy: 88.35%
Test loss: 0.29923 | Test accuracy: 89.14%



In [10]:
model_results = eval_model(
    model=model,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    accuracy_fn=accuracy_fn,
    device=device
)
model_results

{'model_name': 'CNNModel',
 'model_loss': 0.34437206387519836,
 'model_acc': 87.6797124600639}

In [37]:
y_preds = []
model.eval()
with torch.inference_mode():
    for X, y in tqdm(test_dataloader, desc="Making predictions"):
        X, y = X.to(device), y.to(device)
        y_logit = model(X)
        y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1)
        y_preds.append(y_pred.cpu())
y_preds_tensor = torch.cat(y_preds)

Making predictions:   0%|          | 0/313 [00:00<?, ?it/s]

In [36]:
matrix = ConfusionMatrix(num_classes=len(classes), task="multiclass")
matrix_tensor = matrix(preds=y_preds_tensor, target=test_data.targets)

fig, ax = plot_confusion_matrix(
    conf_mat=matrix_tensor.numpy(),
    class_names=classes,
    figsize=(10, 7)
)

RuntimeError: shape '[10, 10]' is invalid for input of size 101